# **Air Quality in Dar es Salaam 🇹🇿**

In [ ]:
# Import libraries here
from pymongo import MongoClient

from pprint import PrettyPrinter

import pandas as pd

import pytz

import matplotlib.pyplot as plt

from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

from sklearn.metrics import mean_absolute_error

from statsmodels.tsa.ar_model import AutoReg

# Prepare Data

In [ ]:
#Connect to MongoDB server

client = MongoClient(host='localhost', port=27017)

In [ ]:
pp = PrettyPrinter(indent=2)

In [ ]:
pp.pprint(list(client.list_databases()))

In [ ]:
db=client['air-quality']
db

In [ ]:
for c in db.list_collections():
    print(c['name'])

In [ ]:
dar=db['dar-es-salaam']

## Explore

In [ ]:
#all the sensor sites in the Dar es Salaam collection

sites = dar.distinct("metadata.site")
sites

In [ ]:
# Determine which site in the Dar es Salaam collection has the most sensor readings 

print (dar.count_documents({'metadata.site':11} ))
print (dar.count_documents({'metadata.site':23} ))

In [ ]:
result =  dar.aggregate(
  
    [{"$group": {"_id": "$metadata.site", "count": {"$count": {}}}}]
)
readings_per_site = list(result)
readings_per_site

## Import

In [ ]:
# create wrangle function

In [ ]:
def wrangle(collection):
    
    results = collection.find(
        {"metadata.site": 11, "metadata.measurement": "P2"},
        projection={"P2": 1, "timestamp": 1, "_id": 0},
    )
    
    y = pd.DataFrame(results).set_index("timestamp")
    
    
    #localize the time-zone
    
    y.index= y.index.tz_localize('UTC').tz_convert("Africa/Dar_es_Salaam")
    
    #remove outliers
    
    y= y[y["P2"] <= 100]
    
    #resample to 1H windows, ffill missing values
    
    y=y["P2"].resample("1H").mean().fillna(method="ffill").to_frame()
    
    
    return y

In [ ]:
y = wrangle(dar)
y.head()

In [ ]:
#Create a time series plot of the readings in y

fig, ax = plt.subplots(figsize=(15, 6))

y["P2"].plot(xlabel= "Date", ylabel="PM2.5 Level", title= "Dar es Salaam PM2.5 Levels" ,ax=ax)

In [ ]:
# Plot the rolling average of the readings in y. Use a window size of 168 (the number of hours in a week)

fig, ax = plt.subplots(figsize=(15, 6))

y["P2"].rolling(168).mean().plot(ax=ax, xlabel= "Date", ylabel="PM2.5 Level", title="Dar es Salaam PM2.5 Levels, 7-Day Rolling Average");

In [ ]:
#Create an ACF plot for the data in y

fig, ax = plt.subplots(figsize=(15, 6))

plot_acf(y, ax=ax)
plt.xlabel('lag [hours]')
plt.ylabel('Correlation coefficient')
plt.title('"Dar es Salaam PM2.5 Readings, ACF"');

In [ ]:
#Create an PACF plot for the data in y

fig, ax = plt.subplots(figsize=(15, 6))

plot_pacf(y, ax=ax)
plt.xlabel('lag [hours]')
plt.ylabel('Correlation coefficient')
plt.title('"Dar es Salaam PM2.5 Readings, PACF"');

## **Split**

In [ ]:
#Split y into training and test sets. The first 90% of the data should be in your training set. The remaining 10% should be in the test set.

cutoff_test = int(len(y)*0.90)

y_train = y.iloc[:cutoff_test].squeeze()
y_test = y.iloc[cutoff_test:].squeeze()

print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

## **Build the model**

In [ ]:
# Establish the baseline mean absolute error of the model.

y_train_mean = y_train.mean()

y_pred_baseline = [y_train_mean]*len(y_train)
mae_baseline = mean_absolute_error(y_train, y_pred_baseline)

print("Mean P2 Reading:", y_train_mean)
print("Baseline MAE:", mae_baseline)

## **Iterate**

In [ ]:
p_params = range(1, 31)
maes = []
for p in p_params:
    
    lags = (p)
    
    model = AutoReg(y_train, lags=lags).fit()
          
    y_pred = model.predict().dropna()
    
    mae = mean_absolute_error(y_train.iloc[p:], y_pred)
    
    maes.append(mae)

    pass
          
 

mae_series = pd.Series(maes, name="mae", index=p_params)
    
mae_series.head()

In [ ]:
#Look through the results in mae_series and determine what value for p provides the best performance
mae_series.sort_values()

In [ ]:
#build and train final_model using the best hyperparameter value.

best_p = 28
best_model = AutoReg(y_train, lags= best_p).fit()

In [ ]:
#Calculate the training residuals for best_model and assign the result to y_train_resid
y_train_resid = best_model.resid
y_train_resid.name = "residuals"
y_train_resid.head()

In [ ]:
# Plot histogram of residuals
y_train_resid.hist()
plt.xlabel("Residuals")
plt.ylabel("Frequency")
plt.title("Best Model, Training Residuals")

In [ ]:
#Create an ACF plot for y_train_resid

fig, ax = plt.subplots(figsize=(15, 6))

plot_acf(y_train_resid ,ax=ax)
plt.xlabel("Lag [hours]")
plt.ylabel("Correlation Coefficient")
plt.title("Dar es Salaam, Training Residuals ACF")

## **Evaluate**

In [ ]:
#Perform walk-forward validation for your model for the entire test set y_test

y_pred_wfv = pd.Series()
history = y_train.copy()
for i in range(len(y_test)):
    
    model = AutoReg(history, lags=28).fit()
    next_pred = model.forecast()
    y_pred_wfv = y_pred_wfv.append(next_pred)
    history = history.append(y_test[next_pred.index])
    
    pass
y_pred_wfv.name = "prediction"
y_pred_wfv.index.name = "timestamp"
y_pred_wfv.head()

## **Communicate Results**

In [ ]:
#plot df_pred_test using plotly express

df_pred_test = ...
fig = ...
fig.update_layout(
    title="Dar es Salaam, WFV Predictions",
    xaxis_title="Date",
    yaxis_title="PM2.5 Level",
)

fig.show()